## Introduction 

This notebook will go over a how to use propensity_scores in mr_uplift packages.The main change is that builds a random forest of the form $t = f(x)$ for the propensity model. It them predicts and calculates the inverse probability as the observation weight:  1/f(x). Note that the propensity model is a multi-classification model and supports more than two treatments. This weight is then fed into both the ERUPT calculations and loss functions of the uplift model. 


In [1]:
import numpy as np
import pytest

from mr_uplift.dataset.data_simulation import get_observational_uplift_data_2
from mr_uplift.mr_uplift import MRUplift, get_t_data
from mr_uplift.keras_model_functionality import prepare_data_optimized_loss
import sys
import pandas as pd



### Data Generating Process

Below is the data generating process of the data. This is similar to the previous data function except that the treatment assignment is a function of the explanatory variable $x_1$. This should simulate a more observational scenario.

\begin{equation}
x_1  \sim runif(0,1)
\end{equation}

\begin{equation}
x_2 \sim runif(0,1)
\end{equation}


\begin{equation}
t \sim rbinom(.2+I(x_1>0)*.6 )
\end{equation}

\begin{equation}
noise \sim rnorm(0,1)
\end{equation}

\begin{equation}
y = x_1*t + noise
\end{equation}


Below the data is generated and two models are built. The first is an uplift model using MSE loss and does not weight observations based on explanatory variables. The second is an uplift model that uses the optimized loss with observations weighted by the propensity model. 

In [2]:
y, x, t = get_observational_uplift_data_2(10000)

y = y[:,0].reshape(-1,1)
uplift_model = MRUplift()
param_grid = dict(num_nodes=[8,64], dropout=[.1, .5], activation=[
                          'relu'], num_layers=[1], epochs=[25], batch_size=[512])

uplift_model.fit(x, y, t.reshape(-1,1), 
                 param_grid = param_grid, n_jobs = 1)

uplift_model_propensity = MRUplift()
param_grid_propensity = dict(num_nodes=[8, 64], dropout=[.1, .5], activation=[
                          'relu'], num_layers=[1], epochs=[25], batch_size=[512],
                 alpha = [.9999,.99], copy_several_times = [1])

uplift_model_propensity.fit(x, y, t.reshape(-1,1), 
                 param_grid = param_grid_propensity, n_jobs = 1,
                optimized_loss = True, use_propensity = True)



In [3]:
#calculate oos predictions 

y_test, x_test, t_test = get_observational_uplift_data_2(10000)

pred_uplift = uplift_model.predict_optimal_treatments(x = x_test)
pred_uplift_propensity = uplift_model_propensity.predict_optimal_treatments(x = x_test, 
                use_propensity_score_cutoff = True)
correct_tmt = (x_test[:,0]>0).reshape(-1,1)*1

FInally the cells below compare the estimated optimal decisions with correct ones for both models. Note that the propensity mdoel correctly identifies optimal treatment for 58% of observations while the standard model only identifies 66% of observations correctly


In [4]:
print('Uplift Model Correct Decisions')
print((pred_uplift == correct_tmt).mean())

print('Uplift Model using Propensity Correct Decisions')
print((pred_uplift_propensity == correct_tmt).mean())

Uplift Model Correct Decisions
0.5754
Uplift Model using Propensity Correct Decisions
0.9886


Finally I calculate the estimated expected responses under propoposed treatments vs actual for both models. 

Without the propensity model the estimate ERUPT is ~.018 which is higher than the actual response under proposed treatments of .06. 

With the propensity model the ERUPT metric is much closer to actual treatments. 

In [5]:
#Estimated uplift vs actual 
print(uplift_model.get_erupt_curves())

print('Actual Estimate of Gains for Uplift Model')
print((pred_uplift*x_test[:,0].reshape(-1,1)).mean())

Using Test Data Set
(       mean       std response_var_names weights assignment treatment
0  0.018898  0.001978              var_0       1      model        -1
0  0.008591  0.002204              var_0       1     random        -1
0 -0.001738  0.001734              var_0      -1        ate       0.0
0  0.150436  0.004449              var_0      -1        ate       1.0,    num_observations  tmt weights  percent_tmt
0              6433  0.0       1        0.919
1               567  1.0       1        0.081)
Actual Estimate of Gains for Uplift Model
0.01450183238033326


In [6]:
#Estimated uplift with propensity vs actual 
print(uplift_model_propensity.get_erupt_curves())

print('Actual Estimate of Gains for Uplift Model with propensity scores')
print((pred_uplift_propensity*x_test[:,0].reshape(-1,1)).mean())

Using Test Data Set
(       mean       std response_var_names weights assignment treatment
0  0.127240  0.002598              var_0       1      model        -1
0  0.001792  0.004851              var_0       1     random        -1
0 -0.001134  0.002104              var_0      -1        ate       0.0
0  0.010204  0.006303              var_0      -1        ate       1.0,    num_observations  tmt weights  percent_tmt
0              3658  1.0       1     0.522571
1              3342  0.0       1     0.477429)
Actual Estimate of Gains for Uplift Model with propensity scores
0.1262029577145106


In [7]:
print('Theoretical Best')
print((correct_tmt*x_test[:,0].reshape(-1,1)).mean())

Theoretical Best
0.12626648857998107
